<a href="https://colab.research.google.com/github/Ibrah-N/PNID_VLM_Qwen2_Training/blob/main/VLM_SLM_Testing_tanzeem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## pdfs to image

In [ ]:
!pip install -q PyMuPDF Pillow

In [ ]:
import fitz               # PyMuPDF
from PIL import Image, ImageFilter
import os
import shutil

def pdf_to_jpg(pdf_path, output_folder="output_images", dpi=300,
               threshold=220, output_folder_bw="bw_imgs", pdf_id=0,
               dir_id=0, blur_radius=0.5, image=False
               ):
    """
    Convert each page of a PDF to:
      • color_image_<page>.jpg
      • bw_image_<page>.jpg

    Args:`
      pdf_path (str): path to .pdf file
      output_folder (str): where to save JPEGs
      dpi (int): rendering resolution
      threshold (0–255): gray cutoff for B/W
      blur_radius (float): Gaussian blur radius on B/W
    """
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(output_folder_bw, exist_ok=True)
    doc = fitz.open(pdf_path)
    page_count = doc.page_count

    for i, page in enumerate(doc, start=1):
        pix = page.get_pixmap(dpi=dpi)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # — Color JPEG
        color_file = os.path.join(output_folder, f"color_image_{i}.jpg")
        img.save(color_file, "JPEG", quality=100, dpi=(dpi, dpi))

        # — Grayscale → threshold → blur → B/W JPEG
        gray = img.convert("L")
        bw = gray.point(lambda x: 255 if x > threshold else 0)
        bw = bw.filter(ImageFilter.GaussianBlur(blur_radius))
        # bw_file = os.path.join(output_folder_bw, f"bw_image_{dir_id}_{pdf_id}_{i}.jpg")
        # bw.save(bw_file, "JPEG", quality=100, dpi=(dpi, dpi))

        return bw

    doc.close()
    print(f"Done! For pdf {pdf_id} Saved {page_count} page(s) to '{output_folder}'")

if __name__ == "__main__":
  pass
  # -- single pdf parse --
  # pdf_path = "/content/PNID_VLM/BOM_PDFs/HEA0748-Combined spares manual 100.pdf"
  # pdf_to_jpg(pdf_path,
              # output_folder="/content/output_images")

"""
  # -- iterate over pdf/img files --
  input_dir = "/content/Equipment patches"
  output_bw = "/content/bw_images/"
  os.makedirs(output_bw, exist_ok=True)


  # os.listdir(input_dir)
  # -- iterate over dir --
  for dirIdx, dir in enumerate(os.listdir(input_dir)):
    if dir.endswith("DS_Store"): continue
    dir_path = os.path.join(input_dir, dir)
    print("Processing direcotry: {}".format(dir_path))
    os.makedirs(os.path.join(output_bw, dir), exist_ok=True)
    output_path = os.path.join(output_bw, dir)

    # -- iterate over sub dir --
    for idx, file_x in enumerate(os.listdir(dir_path)):

      # -- check if pdf --
      if (file_x.endswith(".pdf") or file_x.endswith(".PDF")):
        pdf_path = os.path.join(dir_path, file_x)
        pdf_to_jpg(pdf_path,
                  output_folder=output_path,
                  pdf_id = idx,
                  dir_id = dirIdx,
                  output_folder_bw=output_path)

      # -- check if image --
      elif (file_x.endswith(".jpg") or file_x.endswith(".JPG")):
        src_path = os.path.join(dir_path, file_x)
        dst_path = os.path.join(output_path, file_x)
        print(src_path, dst_path)
        shutil.copy(src_path, dst_path)
        print(f"Done! For :{file_x} Copied to {dst_path}")

      # -- else skip --
      else:
        print("Not a pdf or jpeg file -- file {}".format(file_x))
"""

# Parse Data
  - Verification

## Load Data

In [ ]:
from google.colab import drive
drive.mount("/content/ibrah_drive")

In [ ]:
# Copy the BOM Data Into local drive


import os

global_path = "/content/ibrah_drive/MyDrive/PNID_VLM"
local_path = "PNID_VLM"

os.makedirs(local_path, exist_ok=True)

# -- copy to local path --
!cp -r {global_path}/* {local_path}

In [ ]:
# -- unzip the zipped folder --
!unzip -q {local_path}/BOM_PDFs.zip -d {local_path}
!unzip -q {local_path}/BOM_Output_json.zip -d {local_path}
!unzip -q {local_path}/BOM_CSV_Individual.zip -d {local_path}

In [ ]:
# configs

import os


pdfs_path = "/content/PNID_VLM/BOM_PDFs"
jsons_path = "/content/PNID_VLM/BOM_Outputs"
csvs_path = "/content/PNID_VLM/BOM_CSV_Individual"
new_csvs_path = "/content/PNID_VLM/BOM_CSV_Corrected"

os.makedirs(new_csvs_path, exist_ok=True)

## Parse Sonnet Generetted JSON and Image

In [ ]:
# parse the json date extracted
# directly from the sonnet 4 model

import json
import pandas as pd
import numpy as np
from google.colab.patches import cv2_imshow
pd.set_option('display.max_columns', None)


for idx, pdf in enumerate(os.listdir(pdfs_path)):
  pdf_path = os.path.join(pdfs_path, pdf)
  json_filename = pdf + ".json" # because json name is xyz.pdf.json
  json_path = os.path.join(jsons_path, json_filename)

  print("PDF path: {} - json_path {}".format(pdf_path, json_path))

  # -- if json not exists aginst pdf --
  if not os.path.exists(json_path):
    print(f"Warning: JSON file not found for PDF: {pdf}. Skipping. Expected path: {json_path}")
    continue

  # -- read pdf and json --
  img_data = pdf_to_jpg(pdf_path)
  img_data = np.array(img_data)
  with open(json_path, 'r') as f:
    json_data = json.load(f)
  table_data = pd.DataFrame(json_data['rows'])

  if idx == 0:
    break

PDF path: /content/PNID_VLM/BOM_PDFs/VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 941.pdf - json_path /content/PNID_VLM/BOM_Outputs/VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 941.pdf.json


In [ ]:
cv2_imshow(img_data)

In [ ]:
table_data.head()

## Parse The Converted CSV

In [ ]:
# ---- naming json, csvs and pdfs ----

ashwaryas_csv_naming = os.listdir(csvs_path)
# naming sequence is corrected; now matching with the names of pdf
ashwaryas_corrected_csv_naming = [file_name.replace("_", " ") for file_name in ashwaryas_csv_naming]
vinod_names = os.listdir(pdfs_path)

print("Vinod Names      : ", vinod_names)
print("Ashwaryas Names  : ", ashwaryas_csv_naming)
print("Correct Ash Names: ",ashwaryas_corrected_csv_naming)

Vinod Names      :  ['VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 1049.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 1101.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 929.pdf', 'HEA0748-Combined spares manual 78.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 855.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 885.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 839.pdf', 'HEA0748-Combined spares manual 34.pdf', 'HEA0748-Combined spares manual 98.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 825.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 997.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 1083.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 973.pdf', 'HEA0748-Combined spares manual 110.pdf', 'HEA0748-Combined spares manual 46.pdf', 'VK10 2022-007 Crane Truck Opers - Spare Parts Cat

In [ ]:
# -- move the csvs into new directory --

import shutil

for idx in range(len(ashwaryas_corrected_csv_naming)):
  old_csv_name = ashwaryas_csv_naming[idx] # files_names: are old csv with wrong names
  new_csv_name = ashwaryas_corrected_csv_naming[idx] # correct_file_names: correct names for csvs matching with pdfs

  csv_path = os.path.join(csvs_path, old_csv_name)
  new_csv_path = os.path.join(new_csvs_path, new_csv_name)

  shutil.copy(csv_path, new_csv_path)

In [ ]:
print(sorted(ashwaryas_csv_naming))
print(sorted(ashwaryas_corrected_csv_naming))

['HEA0748-Combined_spares_manual_100.csv', 'HEA0748-Combined_spares_manual_102.csv', 'HEA0748-Combined_spares_manual_104.csv', 'HEA0748-Combined_spares_manual_106.csv', 'HEA0748-Combined_spares_manual_108.csv', 'HEA0748-Combined_spares_manual_11.csv', 'HEA0748-Combined_spares_manual_110.csv', 'HEA0748-Combined_spares_manual_112.csv', 'HEA0748-Combined_spares_manual_114.csv', 'HEA0748-Combined_spares_manual_116.csv', 'HEA0748-Combined_spares_manual_118.csv', 'HEA0748-Combined_spares_manual_120.csv', 'HEA0748-Combined_spares_manual_122.csv', 'HEA0748-Combined_spares_manual_124.csv', 'HEA0748-Combined_spares_manual_13.csv', 'HEA0748-Combined_spares_manual_15.csv', 'HEA0748-Combined_spares_manual_17.csv', 'HEA0748-Combined_spares_manual_19.csv', 'HEA0748-Combined_spares_manual_21.csv', 'HEA0748-Combined_spares_manual_23.csv', 'HEA0748-Combined_spares_manual_25.csv', 'HEA0748-Combined_spares_manual_30.csv', 'HEA0748-Combined_spares_manual_32.csv', 'HEA0748-Combined_spares_manual_34.csv', 'H

In [ ]:
# parse the csv date converted
# into csv for correction

import pandas as pd
import numpy as np
import os
from google.colab.patches import cv2_imshow
pd.set_option('display.max_columns', None)



#======================================#
#   R E A D - C S V - F U N C T I O N  #
#======================================#
def read_csv(csv_path):
  """Read csv of these three formats, utf-8, latin1, cp1252

  Args:
      csv_path (str): path to csv file

  Returns:
      df: pandas dataframe
  """
  try:
      df = pd.read_csv(csv_path, encoding='utf-8')
  except UnicodeDecodeError:
      print("UTF-8 decoding failed, trying 'latin1'...")
      try:
          df = pd.read_csv(csv_path, encoding='latin1')
      except UnicodeDecodeError:
          print("Latin1 decoding also failed, trying 'cp1252'...")
          df = pd.read_csv(csv_path, encoding='cp1252')
  # return
  return df



id = 103
for idx, pdf in enumerate(os.listdir(pdfs_path)):
  pdf_path = os.path.join(pdfs_path, pdf)
  csv_file_name = pdf.replace(".pdf", ".csv") # because json/csv name is xyz.pdf --> xyz.csv
  csv_path = os.path.join(new_csvs_path, csv_file_name)

  if not idx == id:
    continue
  print("PDF path: {} - CSV path {}".format(pdf_path, csv_path))


  # -- if csv not exists aginst pdf --
  if not os.path.exists(csv_path):
    print(f"Warning: CSV file not found for PDF: {pdf}. Skipping. Expected path: {csv_path}")
    continue

  # -- read pdf and csv --
  img_data = pdf_to_jpg(pdf_path)
  img_data = np.array(img_data)
  table_data_df = read_csv(csv_path)


  # Problem: 5th, 6th, 8th-9th, 10th(material specification), 14, 101(not sure, remarks), 102(material specification issue)
  if idx == id:
    break

PDF path: /content/PNID_VLM/BOM_PDFs/VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 837.pdf - CSV path /content/PNID_VLM/BOM_CSV_Corrected/VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 837.csv


In [ ]:
table_data_df.head(20)

In [ ]:
cv2_imshow(img_data)

# Transforming Data
  - path configs
  - drop the column file_name
  - Transforms csvs into jsons

## Utils
  - read_csv function
  - pdf to image function

In [ ]:
!pip install -q PyMuPDF Pillow

In [ ]:
#======================================#
#   R E A D - C S V - F U N C T I O N  #
#======================================#
import pandas as pd

def read_csv(csv_path):
  """Read csv of these three formats, utf-8, latin1, cp1252

  Args:
      csv_path (str): path to csv file

  Returns:
      df: pandas dataframe
  """
  try:
      df = pd.read_csv(csv_path, encoding='utf-8')
  except UnicodeDecodeError:
      print("UTF-8 decoding failed, trying 'latin1'...")
      try:
          df = pd.read_csv(csv_path, encoding='latin1')
      except UnicodeDecodeError:
          print("Latin1 decoding also failed, trying 'cp1252'...")
          df = pd.read_csv(csv_path, encoding='cp1252')
  # return
  return df

In [ ]:
import fitz               # PyMuPDF
from PIL import Image, ImageFilter
import os
import shutil

def pdf_to_jpg(pdf_path, output_folder="output_images", dpi=300,
               threshold=220, output_folder_bw="bw_imgs", pdf_id=0,
               dir_id=0, blur_radius=0.5, image=False
               ):
    """
    Convert each page of a PDF to:
      • color_image_<page>.jpg
      • bw_image_<page>.jpg

    Args:`
      pdf_path (str): path to .pdf file
      output_folder (str): where to save JPEGs
      dpi (int): rendering resolution
      threshold (0–255): gray cutoff for B/W
      blur_radius (float): Gaussian blur radius on B/W
    """
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(output_folder_bw, exist_ok=True)
    doc = fitz.open(pdf_path)
    page_count = doc.page_count

    for i, page in enumerate(doc, start=1):
        pix = page.get_pixmap(dpi=dpi)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # — Color JPEG
        color_file = os.path.join(output_folder, f"color_image_{i}.jpg")
        img.save(color_file, "JPEG", quality=100, dpi=(dpi, dpi))

        # — Grayscale → threshold → blur → B/W JPEG
        gray = img.convert("L")
        bw = gray.point(lambda x: 255 if x > threshold else 0)
        bw = bw.filter(ImageFilter.GaussianBlur(blur_radius))
        # bw_file = os.path.join(output_folder_bw, f"bw_image_{dir_id}_{pdf_id}_{i}.jpg")
        # bw.save(bw_file, "JPEG", quality=100, dpi=(dpi, dpi))

        return bw

    doc.close()
    print(f"Done! For pdf {pdf_id} Saved {page_count} page(s) to '{output_folder}'")

if __name__ == "__main__":
  pass
  # -- single pdf parse --
  # pdf_path = "/content/PNID_VLM/BOM_PDFs/HEA0748-Combined spares manual 100.pdf"
  # pdf_to_jpg(pdf_path,
              # output_folder="/content/output_images")

## Path Configs

In [ ]:
from google.colab import drive
drive.mount("/content/ibrah_drive")

In [ ]:
# -- raw paths configs --

import os

# -- local read configuration --
pdfs_path = "/content/PNID_VLM/BOM_PDFs"
jsons_path = "/content/PNID_VLM/BOM_Outputs"
csvs_path = "/content/PNID_VLM/BOM_CSV_Individual"
new_csvs_path = "/content/PNID_VLM/BOM_CSV_Corrected"

os.makedirs(new_csvs_path, exist_ok=True)


# -- local write configuration --
root_write_path = "VLM_Dataset"
root_write_img_path = os.path.join(root_write_path, "images")
root_write_json_path = os.path.join(root_write_path, "json")

os.makedirs(root_write_img_path, exist_ok=True)
os.makedirs(root_write_json_path, exist_ok=True)

In [ ]:
# ---- naming json, csvs and pdfs ----

ashwaryas_csv_naming = os.listdir(csvs_path)
# naming sequence is corrected; now matching with the names of pdf
ashwaryas_corrected_csv_naming = [file_name.replace("_", " ") for file_name in ashwaryas_csv_naming]
vinod_names = os.listdir(pdfs_path)

print("Vinod Names      : ", vinod_names)
print("Ashwaryas Names  : ", ashwaryas_csv_naming)
print("Correct Ash Names: ",ashwaryas_corrected_csv_naming)

In [ ]:
# -- move the csvs into new directory --

import shutil

for idx in range(len(ashwaryas_corrected_csv_naming)):
  old_csv_name = ashwaryas_csv_naming[idx] # files_names: are old csv with wrong names
  new_csv_name = ashwaryas_corrected_csv_naming[idx] # correct_file_names: correct names for csvs matching with pdfs

  csv_path = os.path.join(csvs_path, old_csv_name)
  new_csv_path = os.path.join(new_csvs_path, new_csv_name)

  shutil.copy(csv_path, new_csv_path)

In [ ]:
#===================================================#
#   C S V - T O - J S O N; P D F - T O - I M A G E  #
#===================================================#
# reads the csv and pdf to jpg
# drop the column file_name
# convert the csv to json
# save each json into new_directory


# -- imports --
import json
import pandas as pd
import os


# -- read csv and pdf --
for pdf_idx, pdf_name in enumerate(os.listdir(pdfs_path)):

  # - pdf and csv names -
  pdf_file_name = pdf_name
  csv_file_name = pdf_name.replace(".pdf", ".csv") # because json/csv name is xyz.pdf --> xyz.csv
  img_file_name = pdf_name.replace(".pdf", ".jpg")
  json_file_name = pdf_name.replace(".pdf", ".json")

  # - read pdf and csv path configuration -
  pdf_path = os.path.join(pdfs_path, pdf_file_name)
  csv_path = os.path.join(new_csvs_path, csv_file_name)

  # - write img and csv configuration -
  img_write_path = os.path.join(root_write_img_path, img_file_name)
  json_write_path = os.path.join(root_write_json_path, json_file_name)


  # - checks conditions -
  if not os.path.exists(csv_path):
    print(f"Warning: CSV file not found for PDF: {pdf_name}. Skipping. Expected path: {csv_path}")
    continue

  # - read pdf and csv -
  img_data = pdf_to_jpg(pdf_path)
  table_data_df = read_csv(csv_path)

  # - transform csv data -
  # drop file name
  # convert nan to ""
  # convert to json
  table_data_df = table_data_df.drop(columns=["file_name", "sheet_name"])
  table_data_df = table_data_df.fillna("")
  table_data_json = '{{"rows" : {}}}'.format(table_data_df.to_json(orient='records'))


  # - write image and json -
  # image
  img_data.save(img_write_path, "JPEG", quality=100, dpi=(300, 300))
  # json
  with open(json_write_path, 'w') as f:
    json.dump(json.loads(table_data_json), f, indent=4)

  print(f"No {pdf_idx} - Image Saved to: {img_write_path} - JSON Saved to: {json_write_path}")


In [ ]:
# copy the VLM_Dataset to drive

!cp /content/VLM_Dataset/* -r /content/ibrah_drive/MyDrive/PNID_VLM/VLM_Dataset/

In [ ]:
# -- GitHub Commits --
# Repo Name: https://github.com/Ibrah-N/PNID_VLM_Qwen2_Training

# COMMITS:
#  1. Data Verification Pipeline. [DONE]
#  2. Data Transformation Pipeline; Converting data from CSVs to JSONs. [DONE]

# Experiments

In [ ]:
import pandas as pd


csv_p = "/content/PNID_VLM/BOM_CSV_Corrected/VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 837.csv"

df = read_csv(csv_p)
df = df.fillna("")
df = df.drop(columns=["file_name", "sheet_name"])
df.head()

In [ ]:
df.to_json("json_xyz.json", orient='records')

In [ ]:
x = df.to_json(orient='records')
json_x = '{{"row" : {}}}'.format(x)
print(json_x)

{"row" : [{"Position Number":1,"Tag Number":"","Description of parts":"Body","Material Specification":"","Quantity":1,"Manufacturer's drawing or Ref.No.":"T1156-143-01","Original Equipment Manufacturer":"Titan","Original Part Manufacturer":"","Manufacturer's real part No.":"T1156-144-01","Unit of Measurement (UOM)":"","Approximate unit price in SAR":"","Recommended Quantity":""},{"Position Number":2,"Tag Number":"","Description of parts":"Circlip","Material Specification":"13x1","Quantity":2,"Manufacturer's drawing or Ref.No.":"T1156-143-01","Original Equipment Manufacturer":"Titan","Original Part Manufacturer":"","Manufacturer's real part No.":"T103-04-012","Unit of Measurement (UOM)":"","Approximate unit price in SAR":"","Recommended Quantity":""},{"Position Number":3,"Tag Number":"","Description of parts":"Castermid","Material Specification":"","Quantity":2,"Manufacturer's drawing or Ref.No.":"T1156-143-01","Original Equipment Manufacturer":"Titan","Original Part Manufacturer":"","M

In [ ]:
import json

# Define the output path for the JSON file
output_json_path = os.path.join(jsons_path, pdf.replace('.pdf', '_corrected.json'))

# Parse the string json_x back to a Python object, then write it to file
with open(output_json_path, 'w') as f:
    json.dump(json.loads(json_x), f, indent=4)

print(f"JSON data saved to: {output_json_path}")

JSON data saved to: /content/PNID_VLM/BOM_Outputs/VK10 2022-007 Crane Truck Opers - Spare Parts Catalogues (1) 837_corrected.json
